In [1]:
import pandas as pd
!pip install numpy pandas
!pip install openpyxl #attribute to install excel reading
%pip install matplotlib seaborn plotly seaborn yfinance
%pip install scikit-learn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from pandas import to_datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#attribute'

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
#reading in all the cleaned dataframes

Inflation_diff = pd.read_csv('Inflation_diff.csv')
Interest_diff = pd.read_csv('Interest_diff.csv')
Current_account_diff = pd.read_csv('Current_account_diff.csv')
swaps_df = pd.read_csv('merged_df_month_year.csv')

In [3]:
#now run the regressiosn since all the data is ready, explore !!!!!

In [4]:
#checking if the dataframes have the same number of rows
print(f"Inflation_diff rows: {len(Inflation_diff)}")
print(f"Interest_diff rows: {len(Interest_diff)}")
print(f"Current_account_diff rows: {len(Current_account_diff)}")
print(f"swaps_df rows: {len(swaps_df)}")


Inflation_diff rows: 235
Interest_diff rows: 235
Current_account_diff rows: 235
swaps_df rows: 235


In [5]:
#lets check the first and last dates
print(f"Inflation_diff first date: {Inflation_diff['Date'].min()}")
print(f"Inflation_diff last date: {Inflation_diff['Date'].max()}")
print(f"Interest_diff first date: {Interest_diff['Date'].min()}")
print(f"Interest_diff last date: {Interest_diff['Date'].max()}")
print(f"Current_account_diff first date: {Current_account_diff['Date'].min()}")
print(f"Current_account_diff last date: {Current_account_diff['Date'].max()}")
print(f"swaps_df first date: {swaps_df['month-year'].min()}")
print(f"swaps_df last date: {swaps_df['month-year'].max()}")
#now we see that the swaps_df has dates, lets change it to month_year format. 

Inflation_diff first date: 2006-01
Inflation_diff last date: 2025-07
Interest_diff first date: 2006-01
Interest_diff last date: 2025-07
Current_account_diff first date: 2006-01
Current_account_diff last date: 2025-07
swaps_df first date: 2006-01-01
swaps_df last date: 2025-07-01


In [21]:
#lets create a function to divide the dataframe into train and test sets, based on the date
def train_test_split_by_date(df, date_column, test_size=0.2):
    """
    Splits the dataframe into train and test sets based on the date.
    
    Parameters:
    df (DataFrame): The dataframe to split.
    date_column (str): The name of the date column.
    test_size (float): The proportion of the dataset to include in the test split.
    
    Returns:
    DataFrame: Train set.
    DataFrame: Test set.
    """
    df[date_column] = pd.to_datetime(df[date_column])
    cutoff_date = df[date_column].quantile(1 - test_size)
    
    train_set = df[df[date_column] < cutoff_date]
    test_set = df[df[date_column] >= cutoff_date]
    
    return train_set, test_set

train_test_split_by_date(Inflation_diff, 'Date')
train_set_inflation, test_set_inflation = train_test_split_by_date(Inflation_diff, 'Date')

train_test_split_by_date(Interest_diff, 'Date')
train_set_interest, test_set_interest = train_test_split_by_date(Interest_diff, 'Date')
train_test_split_by_date(Current_account_diff, 'Date')
train_set_current, test_set_current = train_test_split_by_date(Current_account_diff, 'Date')
train_test_split_by_date(swaps_df, 'month-year')
train_set_swaps, test_set_swaps = train_test_split_by_date(swaps_df, 'month-year')


In [ ]:
#running the regression model for USDJPY-1 factor which is inflation
#train values
USDJPY_train = train_set_swaps['USDJPY_20Y'].pct_change().dropna()
X_inflation_USDJPY_train= train_set_inflation['USDJPY']
X_inflation_USDJPY_train= X_inflation_USDJPY_train.values.reshape(-1, 1)
y_inflation_USDJPY_train = USDJPY_train.values.reshape(-1, 1)
model_inflation_USDJPY = LinearRegression()
model_inflation_USDJPY.fit(X_inflation_USDJPY_train, y_inflation_USDJPY_train)
#test values
USDJPY_test = test_set_swaps['USDJPY_20Y'].pct_change().dropna()
X_inflation_USDJPY_test = test_set_inflation['USDJPY']
X_inflation_USDJPY_test = X_inflation_USDJPY_test.values.reshape(-1, 1)
y_inflation_USDJPY_test = USDJPY_test.values.reshape(-1, 1)
#align everything by index
aligned_index
print(model_inflation_USDJPY.score(X_inflation_USDJPY_test, y_inflation_USDJPY_test))
print(model_inflation_USDJPY.score(X_inflation_USDJPY_train, y_inflation_USDJPY_train))


ValueError: Found input variables with inconsistent numbers of samples: [188, 187]

poor fitting to test values, lets figure out our next steps

In [26]:


# --- TRAIN SET ---
# Calculate % change for both X and y, then align
X_train = train_set_inflation['USDJPY'].pct_change()
y_train = train_set_swaps['USDJPY_20Y'].pct_change()

# Combine into a single DataFrame to keep them aligned
train_df = pd.concat([X_train, y_train], axis=1).dropna()

# Split back into arrays
X_inflation_USDJPY_train = train_df.iloc[:, 0].values.reshape(-1, 1)
y_inflation_USDJPY_train = train_df.iloc[:, 1].values.reshape(-1, 1)

# Fit model
model_inflation_USDJPY = LinearRegression()
model_inflation_USDJPY.fit(X_inflation_USDJPY_train, y_inflation_USDJPY_train)


# --- TEST SET ---
X_test = test_set_inflation['USDJPY'].pct_change()
y_test = test_set_swaps['USDJPY_20Y'].pct_change()

test_df = pd.concat([X_test, y_test], axis=1).dropna()

X_inflation_USDJPY_test = test_df.iloc[:, 0].values.reshape(-1, 1)
y_inflation_USDJPY_test = test_df.iloc[:, 1].values.reshape(-1, 1)

# Predict & score
y_pred = model_inflation_USDJPY.predict(X_inflation_USDJPY_test)

print(f"Train R²: {model_inflation_USDJPY.score(X_inflation_USDJPY_train, y_inflation_USDJPY_train):.4f}")
print(f"Test R²: {model_inflation_USDJPY.score(X_inflation_USDJPY_test, y_inflation_USDJPY_test):.4f}")


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [27]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd

# --- TRAIN SET ---
X_train = train_set_inflation['USDJPY'].pct_change()
y_train = train_set_swaps['USDJPY_20Y'].pct_change()

# Replace inf/-inf with NaN
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
y_train.replace([np.inf, -np.inf], np.nan, inplace=True)

# Combine and drop NaN
train_df = pd.concat([X_train, y_train], axis=1).dropna()

# Split back into arrays
X_inflation_USDJPY_train = train_df.iloc[:, 0].values.reshape(-1, 1)
y_inflation_USDJPY_train = train_df.iloc[:, 1].values.reshape(-1, 1)

# --- TEST SET ---
X_test = test_set_inflation['USDJPY'].pct_change()
y_test = test_set_swaps['USDJPY_20Y'].pct_change()

X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
y_test.replace([np.inf, -np.inf], np.nan, inplace=True)

test_df = pd.concat([X_test, y_test], axis=1).dropna()

X_inflation_USDJPY_test = test_df.iloc[:, 0].values.reshape(-1, 1)
y_inflation_USDJPY_test = test_df.iloc[:, 1].values.reshape(-1, 1)

# Fit model
model_inflation_USDJPY = LinearRegression()
model_inflation_USDJPY.fit(X_inflation_USDJPY_train, y_inflation_USDJPY_train)

# Predict & score
y_pred = model_inflation_USDJPY.predict(X_inflation_USDJPY_test)

print(f"Train R²: {model_inflation_USDJPY.score(X_inflation_USDJPY_train, y_inflation_USDJPY_train):.4f}")
print(f"Test R²: {model_inflation_USDJPY.score(X_inflation_USDJPY_test, y_inflation_USDJPY_test):.4f}")


Train R²: 0.0004
Test R²: -0.0646


In [ ]:
#lets implement 3 factor model for USDJPY before train test split and after train test split
#before train test split